In [1]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load pre-trained models
fer_model = load_model('best_fer_model.keras')  # Facial Expression Recognition model
utk_model = load_model('best_utk_model.keras')  # Age, Gender, and Ethnicity model

# Function for real-time prediction
def predict_attributes(fer_model, utk_model, frame):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)  # Detect faces

    for (x, y, w, h) in faces:
        # Crop and preprocess the detected face
        face = gray[y:y+h, x:x+w]
        face = cv2.resize(face, (48, 48))
        face = face.astype('float32') / 255.0
        face = np.expand_dims(face, axis=-1)
        face = np.expand_dims(face, axis=0)

        # Predict facial expression
        expression_pred = fer_model.predict(face, verbose=0)
        expression_label = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'][np.argmax(expression_pred)]

        # Predict age, gender, and ethnicity
        age_pred, gender_pred, ethnicity_pred = utk_model.predict(face, verbose=0)
        age = int(age_pred[0][0])
        gender = 'Male' if gender_pred[0][0] > 0.5 else 'Female'
        ethnicity_label = ['White', 'Black', 'Asian', 'Indian', 'Others'][np.argmax(ethnicity_pred)]

        # Draw rectangle around face
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Display predictions
        cv2.putText(frame, f"Expr: {expression_label}", (x, y-60), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)
        cv2.putText(frame, f"Age: {age}", (x, y-40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)
        cv2.putText(frame, f"Gender: {gender}", (x, y-20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)
        cv2.putText(frame, f"Ethnicity: {ethnicity_label}", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (36, 255, 12), 2)

    return frame

# Main function for real-time video processing
def main():
    # Initialize video capture
    cap = cv2.VideoCapture(0)  # Use default camera

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Process the frame and make predictions
        processed_frame = predict_attributes(fer_model, utk_model, frame)

        # Display the processed frame
        cv2.imshow('Facial Attribute Detection', processed_frame)

        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()

# Run the application
if __name__ == "__main__":
    main()
